# Day 15

Switching to a notebook for this one as it will be easier to get a measure of the problem. First load the data. 

In [8]:
import parse

def parse_data(s):
    sensors = []
    for line in s.strip().splitlines():
        r = parse.parse(
            "Sensor at x={sensor_x:d}, y={sensor_y:d}: closest beacon is at x={beacon_x:d}, y={beacon_y:d}",
            line
        )
        sensors.append(((r["sensor_x"], r["sensor_y"]), (r["beacon_x"], r["beacon_y"])))
    return sensors

test_sensors = parse_data("""
Sensor at x=2, y=18: closest beacon is at x=-2, y=15
Sensor at x=9, y=16: closest beacon is at x=10, y=16
Sensor at x=13, y=2: closest beacon is at x=15, y=3
Sensor at x=12, y=14: closest beacon is at x=10, y=16
Sensor at x=10, y=20: closest beacon is at x=10, y=16
Sensor at x=14, y=17: closest beacon is at x=10, y=16
Sensor at x=8, y=7: closest beacon is at x=2, y=10
Sensor at x=2, y=0: closest beacon is at x=2, y=10
Sensor at x=0, y=11: closest beacon is at x=2, y=10
Sensor at x=20, y=14: closest beacon is at x=25, y=17
Sensor at x=17, y=20: closest beacon is at x=21, y=22
Sensor at x=16, y=7: closest beacon is at x=15, y=3
Sensor at x=14, y=3: closest beacon is at x=15, y=3
Sensor at x=20, y=1: closest beacon is at x=15, y=3
""")

test_sensors

[((2, 18), (-2, 15)),
 ((9, 16), (10, 16)),
 ((13, 2), (15, 3)),
 ((12, 14), (10, 16)),
 ((10, 20), (10, 16)),
 ((14, 17), (10, 16)),
 ((8, 7), (2, 10)),
 ((2, 0), (2, 10)),
 ((0, 11), (2, 10)),
 ((20, 14), (25, 17)),
 ((17, 20), (21, 22)),
 ((16, 7), (15, 3)),
 ((14, 3), (15, 3)),
 ((20, 1), (15, 3))]

In [10]:
from itertools import chain

sensors = parse_data(open("input").read())

xs = list(chain(*((s[0][0], s[1][0]) for s in sensors)))
ys = list(chain(*((s[0][1], s[1][1]) for s in sensors)))

x_range = max(xs) - min(xs)
y_range = max(ys) - min(ys)
print(f"{x_range} * {y_range} = {x_range * y_range}")

3818017 * 4081722 = 15584083985274


That's not going to be held in memory so an array's out of the question. How big are the distances? Could we generate sets of sensed positions?

In [13]:
def manhattan_distance(point_a, point_b):
    a_x, a_y = point_a
    b_x, b_y = point_b
    return abs(a_x - b_x) + abs(a_y - b_y)

manhattans = [manhattan_distance(s, b) for s, b in sensors]
max(manhattans)

1908498

I think that's too big, this will have to be tackled by calculating ranges of sensed positions on the row for each sensor.

In [21]:
def range(y, sensor, beacon):
    manhattan = manhattan_distance(sensor, beacon)
    sensor_x, sensor_y = sensor
    x_distance_at_y = manhattan - abs(sensor_y - y)
    if x_distance_at_y >= 0:
        yield (sensor_x - x_distance_at_y, sensor_x + x_distance_at_y)
        
next(range(10, (8, 7), (2, 10)))

(2, 14)

In [24]:
def ranges(y, sensors):
    for s, b in sensors:
        yield from range(y, s, b)
        
list(ranges(10, test_sensors))

[(12, 12), (2, 14), (2, 2), (-2, 2), (16, 24), (14, 18)]

That gives a range of $24 - (-2) + 1 = 27$, which is one more than in the instructions. Let's check if the existing beacon needs to be excluded.

In [25]:
s = "..####B######################.."
s.count('#')

26

Yes it does. Now to reduce overlapping ranges.

In [ ]:
def overlap(p1, p2):
    p1.x, p1.y = p1
    p2.x, p2.y = p2
    return p2.x <= p1.x <= p2.y or p1.x <= p2.x <= p1.y

